In [19]:
pip install -q langchain langchain-community chromadb sentence-transformers openai tiktoken python-dotenv langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI

# DO NOT EXPOSE THIS KEY PUBLICLY!!!!
api_key = "sk-..."
# DO NOT EXPOSE THIS KEY PUBLICLY!!!!

project_id = "proj_fHRnVJY0Oyfm1ufG1sffxa6W"

client = OpenAI(api_key=api_key, project=project_id)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

In [23]:
import pandas as pd
df = pd.read_csv('..\Data\Student_rubric_feedback.csv', skipinitialspace=True)
df.head()

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\colet\AppData\Local\Temp\ipykernel_35648\4264941539.py:2: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_csv('..\Data\Student_rubric_feedback.csv', skipinitialspace=True)


,Student,Criteria,Ratings,Points,Points Awarded,Final Grade,Feedback
0,Student_1,Functionality,Code completes without errors,25,25,100,Excellent job! Your code runs without errors a...
1,Student_1,Correctness/Style,Code is stylistically correct and works as pla...,25,25,100,Excellent job! Your code runs without errors a...
2,Student_1,Analytical Detail,Analysis includes appropriate statistical test...,25,25,100,Excellent job! Your code runs without errors a...
3,Student_1,Checkpoint Completion,All checkpoints were submitted on time.,25,25,100,Excellent job! Your code runs without errors a...
4,Student_2,Functionality,Code completes without errors,25,18,67,Minor errors noted but your code runs for the ...


In [24]:
# coulmn names have trailing whitespace
df.columns = df.columns.str.strip()
expected = ["Student", "Criteria", "Ratings", "Points", "Points Awarded", "Final Grade", "Feedback"]
missing = [c for c in expected if c not in df.columns]
if missing:
  raise ValueError(f"Missing expected columns after normalization: {missing}")

# --- Clean text columns ---
for col in ["Student", "Criteria", "Ratings", "Feedback"]:
  df[col] = df[col].astype(str).str.strip()

# --- Convert numeric columns ---
df["Points"] = pd.to_numeric(df["Points"], errors="coerce").fillna(25)
df["Points Awarded"] = pd.to_numeric(df["Points Awarded"], errors="coerce").fillna(0)
df["Final Grade"] = pd.to_numeric(df["Final Grade"], errors="coerce")

# --- Add unique ID for each row ---
df["id"] = [f"row-{i}" for i in range(len(df))]

In [25]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Use raw strings or forward slashes on Windows to avoid escape issues
pdf_folder = [
    "..\Data\SQL slides\Advanced SQL I copy.pdf",
    "..\Data\SQL slides\Advanced SQL II copy.pptx.pdf",
    "..\Data\Assignment_prompt.pdf",
    "..\Data\Syllabus.pdf",
]


"""
all_docs = []
# --- Load and add metadata ---
for file in pdf_folder:
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()

    # add source metadata for each page
    for p in pages:
        p.metadata["source_name"] = file

    all_docs.extend(pages)  # collect everything in one list

embeddings = OpenAIEmbeddings()
"""
pdf_docs = {}

for file in pdf_folder:
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    # add source metadata for each page
    for p in pages:
        p.metadata["source_name"] = file
    # store all pages for this file
    pdf_docs[file] = pages

# Flatten dictionary into one list of docs
flat_docs = [doc for pages in pdf_docs.values() for doc in pages]

embeddings = OpenAIEmbeddings()

#persist_dir = "unified_chroma_db"

<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:11: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:11: SyntaxWarning: invalid escape sequence '\D'
C:\Users\colet\AppData\Local\Temp\ipykernel_35648\734577981.py:8: SyntaxWarning: invalid escape sequence '\D'
  "..\Data\SQL slides\Advanced SQL I copy.pdf",
C:\Users\colet\AppData\Local\Temp\ipykernel_35648\734577981.py:9: SyntaxWarning: invalid escape sequence '\D'
  "..\Data\SQL slides\Advanced SQL II copy.pptx.pdf",
C:\Users\colet\AppData\Local\Temp\ipykernel_35648\734577981.py:10: SyntaxWarning: invalid escape sequence '\D'
  "..\Data\Assignment_prompt.pdf",
C:\Users\colet\AppData\Local\Temp\ipykernel_35648\734577981.py:11: SyntaxWarning: invalid escape sequence '\D'
  "..\Data\Syllab

In [26]:
#vectordb = Chroma.from_documents(documents=all_docs, embedding=embeddings, persist_directory="./vectordb_index")
#vectordb.persist()
# 2. Create Chroma DB from flat_docs
vectordb = Chroma.from_documents(
    documents=flat_docs,
    embedding= embeddings,
    persist_directory="./vectordb_index"
)
vectordb.persist()

C:\Users\colet\AppData\Local\Temp\ipykernel_35648\2228153208.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [27]:
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [28]:
for i, doc in enumerate(flat_docs[:5]):
    print(f"\n--- Raw Doc {i+1} from {doc.metadata['source_name']} ---")
    print(doc.page_content[:500])  # peek at first 500 chars


--- Raw Doc 1 from ..\Data\SQL slides\Advanced SQL I copy.pdf ---
Advanced SQL I

--- Raw Doc 2 from ..\Data\SQL slides\Advanced SQL I copy.pdf ---
Agenda - Schedule
1. SQL Leetcode Q
2. Window Functions
3. Break
4. COVID-19 SQL Lab
 Database systems of the past

--- Raw Doc 3 from ..\Data\SQL slides\Advanced SQL I copy.pdf ---
Agenda - Announcements 
● No pre-class quiz
● TLAB #3 due 5/14
○ Early grade due date: 5/7
○ Extension due date: 5/13
● In-class end of phase project is being released THIS THURSDAY! (We 
recommend attending this review session)

--- Raw Doc 4 from ..\Data\SQL slides\Advanced SQL I copy.pdf ---
Agenda - Goals
● Use SQL window functions (ROW_NUMBER, RANK, DENSE_RANK, NTILE) to 
analyze and rank data within partitions.
● Apply LAG() and LEAD() functions to compare rows across a sequence.
● Use subqueries to calculate differences between values (e.g., current vs. 
previous order amounts).

--- Raw Doc 5 from ..\Data\SQL slides\Advanced SQL I copy.pdf ---
SQL Leetc

In [29]:
for file in pdf_folder:
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    print(f"{file}: {len(pages)} pages")
    print(f"First page text: {pages[0].page_content[:500]}")

..\Data\SQL slides\Advanced SQL I copy.pdf: 43 pages
First page text: Advanced SQL I
..\Data\SQL slides\Advanced SQL II copy.pptx.pdf: 41 pages
First page text: Advanced SQL II
..\Data\Assignment_prompt.pdf: 2 pages
First page text: Ad-hoc  Analysis  
You  are  a  data  analyst  at  an  NYC-based  wholesale  food  supplier  called  Osiris-Foods.  As  part  of  
your
 
companies
 
mission
 
to
 
make
 
more
 
data-driven
 
decisions,
 
you
 
are
 
tasked
 
with
 
generating
 
an
 
ad-hoc
 
report
 
on
 
your
 
companies
 
database.
 
You
 
will
 
report
 
on
 
descriptive
 
statistics
 
using
 
SQL
 
and
 
generate
 
respective
 
visualizations
 
using
 
pandas
 
to
 
provide
 
your
 
company
 
an
 
overview
 
of
 
its'
 
pa
..\Data\Syllabus.pdf: 13 pages
First page text: Innovation  Fellowship
 Data  Science  Syllabus   Class  Summary:  The  Knowledge  House  Data  Science  Innovation  Fellowship  aims  to  provide  students  with  a  multifaceted,  
foundational
 
skill
 
set
 
essent

In [30]:
# 🔎 Try a query against your retriever
query = "What is the ad-hoc analysis assignment_prompt?"
docs = vector_retriever.get_relevant_documents(query)

# Print results with source info + preview
for i, doc in enumerate(docs, 1):
    source = doc.metadata.get("source_name", "Unknown source")
    print(f"\n--- Document {i} (from {source}) ---")
    print(doc.page_content[:500])


C:\Users\colet\AppData\Local\Temp\ipykernel_35648\4163857892.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vector_retriever.get_relevant_documents(query)



--- Document 1 (from ..\Data\Assignment_prompt.pdf) ---
Ad-hoc  Analysis  
You  are  a  data  analyst  at  an  NYC-based  wholesale  food  supplier  called  Osiris-Foods.  As  part  of  
your
 
companies
 
mission
 
to
 
make
 
more
 
data-driven
 
decisions,
 
you
 
are
 
tasked
 
with
 
generating
 
an
 
ad-hoc
 
report
 
on
 
your
 
companies
 
database.
 
You
 
will
 
report
 
on
 
descriptive
 
statistics
 
using
 
SQL
 
and
 
generate
 
respective
 
visualizations
 
using
 
pandas
 
to
 
provide
 
your
 
company
 
an
 
overview
 
of
 
its'
 
pa

--- Document 2 (from ..\Data\SQL slides\Advanced SQL II copy.pptx.pdf) ---
Advanced SQL II

--- Document 3 (from ..\Data\SQL slides\Advanced SQL I copy.pdf) ---
SQL Review


In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

In [32]:
feedback = df.groupby('Student')["Feedback"].unique().apply(list)
feedback

Student
Student_1     [Excellent job! Your code runs without errors ...
Student_10    [Functional solution, but missed checkpoints h...
Student_2     [Minor errors noted but your code runs for the...
Student_3                   [Good try. Improvements to be made]
Student_4     [Your SQL code runs overall, but a few minor i...
Student_5     [Your submission was complete and code execute...
Student_6     [Code was incomplete however assignment was su...
Student_7                                          [Well done.]
Student_8     [You demonstrated consistency across all areas...
Student_9     [Your analysis was strong and very thorough, b...
Name: Feedback, dtype: object

In [39]:
# AI Agent: Feedback Analyzer which will do an analysis of the feedback they received.
# Check the quality of the feedback (vague, expansive).
def feedback_analyzer(feedback: list) -> list:
  #Grab context for the feedback
  docs = vector_retriever.get_relevant_documents("What grading criteria are used for SQL assignments??")
  #Create a prompt for Agent Feedback Analyzer
  prompt = f"""
    You are Agent Feedback Analyzer.
    Your job is to determine the quality of the instructor feedback per student.
    The feedback is expansive if it includes specfic information.
    Example of expansive feed back is "Your SQL code runs overall, but a few minor issues should be addressed. For example, you missed some join statements.These are small fixes and don’t indicate a major bug. Queries are well-structured and stylistically strong. The analysis includes appropriate tests, metrics, and clear visuals, showing solid reasoning. Be sure to submit all checkpoints on time to stay fully on track."
    The feedback is vague of it has general information.
    Example of vague feed back: "Good try. Improvements to be made"


    Example Output:
    Student_1's {feedback} is vague
    Student_2's {feedback} is moderate
    Student_3's {feedback} is expansive.

    Output the results as a dictionary: 'Student 1: vague'
    """

  # get back answer from `gpt-4o-mini` using context & prompt
  resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
  #return resp.choices[0].message.content
  #return resp.choices[0].message.content.split()
  #cleaned = resp.choices[0].message.content.replace("\\n", "\n")
  cleaned = resp.choices[0].message.content.strip()
  #print(cleaned)
  return cleaned


In [40]:
#Running the feedback Analyzer function
fa_results = feedback_analyzer(feedback)
fa_results

"```python\n{\n    'Student 1': 'expansive',\n    'Student 2': 'moderate',\n    'Student 3': 'vague',\n    'Student 4': 'expansive',\n    'Student 5': 'expansive',\n    'Student 6': 'vague',\n    'Student 7': 'vague',\n    'Student 8': 'expansive',\n    'Student 9': 'expansive',\n    'Student 10': 'moderate'\n}\n```"

In [36]:
#AI Agent WeakPoints
  #this AI agent shows the weakpoints or strong points the person needs to study
  #if feedback is vague -> relying on rubric rating + criteria, and assignment prompt to establish weakpoints
  #if feedback is moderate -> relying on rubric rating + criteria, assignment prompt, and instructor feedback to establish weakpoints
  #if feedback is expansive -> relying on rubric rating + criteria, assignment prompt, and more of the instructor feedback to establish weakpoints

#Process
  # Student 3's feedback was vague: "Good try. Improvements to be made" ->
  # *looks at rubric rating* "Analysis includes some visualizations or metrics but lacks depth or completeness."
  # *looks at criteria*:"Analytical Detail: If applicable, is visual analysis of the data necessary and reasonable? Fellow must demonstrate not only technical ability in generating analysis but also the ability to describe their reasoning for said analysis. For instance, do we include the necessary statistical tests, model measurements, and visual exploratory data analysis which can provide background context and insight into performance? These can include things like ANOVA, measures of central tendency, heatmaps, bar graphs, model metrics, and so on."
  # *Looks at  assignment prompt*

#Output (Return Weakpoints)
  # List of WEAKPOINTS -> "Did not include a join between sales and product categories; analysis is incomplete without category insights.-->

In [37]:
def Weakpoint_Detector(fa_results: dict) -> str:

  #Grab context for the weakpoints
  context = vector_retriever.get_relevant_documents("What is the ad-hoc analysis assignment_prompt?")

  #Create a prompt for Agent Weakpoint Detector
  prompt = f"""
  You are Agent Weakpoint Detector. You job is to utilize the {fa_results} and connect to the {context} to create weakpoints that a user will need to study
  If feedback is vague -> relying on rubric rating + criteria, and assignment prompt to establish weakpoints
  If feedback is moderate -> relying on rubric rating + criteria, assignment prompt, and instructor feedback to establish weakpoints
  If feedback is expansive -> relying on rubric rating + criteria, assignment prompt, and more of the instructor feedback to establish weakpoints
  """
  # get back answer from `gpt-4o-mini` using context & prompt
  resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
  #return resp.choices[0].message.content
  #return resp.choices[0].message.content.split()
  #cleaned = resp.choices[0].message.content.replace("\\n", "\n")
  cleaned = resp.choices[0].message.content.strip()
  #print(cleaned)
  return cleaned



In [38]:
Weakpoint_Detector(fa_results)

'Based on the classifications of student feedback and the provided assignment prompt, here are the identified weakpoints for each student that they may need to study:\n\n### Weakpoints for Each Student\n\n- **Student 1 (Expansive)**:\n  - No weakpoints identified; feedback was detailed and specific.\n\n- **Student 2 (Vague)**:\n  - **Weakpoint 1**: Lack of specific details about missed checkpoints. Students should focus on understanding the assignment requirements and ensuring they address all checkpoints.\n  - **Weakpoint 2**: Need to improve the ability to provide constructive feedback. This includes learning how to identify specific areas of improvement in code or analysis.\n\n- **Student 3 (Moderate)**:\n  - **Weakpoint 1**: Need to identify and explain minor errors in detail. Students should practice articulating specific issues in their work.\n  - **Weakpoint 2**: Understanding the importance of meeting all requirements. Review the assignment prompt to ensure all aspects are cove